In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests as req
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
  # 웹드라이버 실행
driver = webdriver.Chrome()

# 제품 목록 페이지 로드
driver.get("https://www.flipkart.com/air-conditioners/pr?sid=j9e,abm,c54&p[]=facets.fulfilled_by%255B%255D%3DFlipkart%2BAssured&p[]=facets.brand%255B%255D%3DLG&p[]=facets.serviceability%5B%5D%3Dtrue&otracker=categorytree&otracker=nmenu_sub_TVs%20%26%20Appliances_0_LG")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CGtC98')))

# URL을 저장할 리스트 초기화
url_list = []

# 요소 찾기
elements = driver.find_elements(By.CLASS_NAME, 'CGtC98')

# 각 요소에서 'href' 속성을 가져와 리스트에 추가 (sponsored 요소는 제외)
for element in elements:
    try:
        # 'f8qK5m' 클래스를 가진 자식 요소가 있는지 확인
        if element.find_elements(By.CLASS_NAME, 'f8qK5m'):
            # 'f8qK5m' 클래스를 가진 자식 요소가 있다면 skip
            continue
        # 자식 요소가 'f8qK5m' 클래스를 가지고 있지 않다면 href 추출
        url_value = element.get_attribute('href')
        if url_value:  # href 속성이 있는 경우만 추가
            url_list.append(url_value)
    except NoSuchElementException:
        # 자식 요소가 없으면 그냥 넘어가기
        continue 
        
# 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

for url in tqdm(url_list):
    try:
        driver.get(url)
        
        # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용...
        try:
            # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용... 
            list_items = driver.find_elements(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
            
            # 텍스트 추출
            hp_data = [item.text for item in list_items]
            if not hp_data:
                hp_data = ["No data"]  # 데이터가 없을 경우 기본값
        except (NoSuchElementException, TimeoutException) as e:
            hp_data = ["Error fetching data"]  # 오류 발생 시 기본값
        
        # 리뷰 버튼 클릭 대기
        try:
            review_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_23J90q.RcXBOT")))
            if review_button.is_displayed():
                review_button.click()
        except (NoSuchElementException, TimeoutException):
            pass
        
        # 낮은 별점 접근
        try:
            low_rating_option = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > div > div.VLIitu > div > div > div.DOjaWF.gdgoEp.col-9-12 > div.DOjaWF.gdgoEp.col-12-12 > div > div:nth-child(2) > div > div > div > div > select > option:nth-child(4)")))
            low_rating_option.click()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error selecting sort option for URL {url}: {e}")
        
        # 데이터 확보
        page_number = 1
        
        while True:
            try:
                # 페이지 URL 업데이트
                current_url = driver.current_url
                full_url = f'{current_url}&page={page_number}'
                driver.get(full_url)
                
                # ZmyHeo 클래스의 존재 여부를 확인
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZmyHeo')))
                
                # 데이터 추출
                text_elements = driver.find_elements(By.CLASS_NAME, 'ZmyHeo')
                state_elements = driver.find_elements(By.CLASS_NAME, 'MztJPv')
                date_elements = driver.find_elements(By.CLASS_NAME, '_2NsDsF')
                star_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'XQDdHH') and (contains(@class, 'Js30Fc') or contains(@class, 'Czs3gR') or contains(@class, 'Ga3i8K'))]")
                
                text_list = [elem.text for elem in text_elements]
                state_list = [elem.text for elem in state_elements]
                date_list = [elem.text for elem in date_elements]
                dates = date_list[1::2]
                star_list = [elem.text for elem in star_elements]
                
                # 리뷰 데이터를 모든 리뷰 리스트에 추가
                for t, s, d, r in zip(text_list, state_list, dates, star_list):
                    for data in hp_data:
                        all_reviews.append({
                            'text': t,
                            'state': s,
                            'date': d,
                            'star': r,
                            'hp_data': data  # 각각의 hp_data 항목을 추가
                        })

                # 페이지 번호 증가
                page_number += 1
                
            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error on page {page_number} for URL {full_url}: {e}")
                break
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# 현재 창 닫기
driver.quit()

  8%|██████▍                                                                            | 1/13 [00:29<05:50, 29.18s/it]

Error processing URL https://www.flipkart.com/lg-ai-convertible-6-in-1-cooling-2024-model-1-5-ton-3-star-split-dual-inverter-hd-filter-anti-virus-protection-viraat-mode-diet-mode-adc-sensor-ac-white/p/itm1aaacb6452e5a?pid=ACNGX677RWXAXJRX&lid=LSTACNGX677RWXAXJRXINO67L&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&spotlightTagId=FkPickId_j9e%2Fabm%2Fc54&srno=b_1_4&otracker=browse&iid=10ac34b5-4af4-4100-a1aa-d13ee6c44a86.ACNGX677RWXAXJRX.SEARCH&ssid=yd80e3fvb40000001724715609163: name 'NoSuchElementException' is not defined


 15%|████████████▊                                                                      | 2/13 [00:57<05:16, 28.80s/it]

Error processing URL https://www.flipkart.com/lg-ai-convertible-6-in-1-cooling-2024-model-1-5-ton-5-star-split-inverter-4-way-swing-hd-filter-anti-virus-protection-ac-viraat-mode-adc-sensor-white/p/itm0e19df12f607b?pid=ACNGWCNRAQTHEV5Z&lid=LSTACNGWCNRAQTHEV5ZSU353L&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_5&otracker=browse&iid=10ac34b5-4af4-4100-a1aa-d13ee6c44a86.ACNGWCNRAQTHEV5Z.SEARCH&ssid=yd80e3fvb40000001724715609163: name 'NoSuchElementException' is not defined


 23%|███████████████████▏                                                               | 3/13 [01:25<04:42, 28.29s/it]

...

Error processing URL https://www.flipkart.com/lg-ai-convertible-6-in-1-cooling-2024-model-1-5-ton-5-star-split-dual-inverter-4-way-swing-hd-filter-anti-virus-protection-viraat-mode-adc-sensor-ac-white/p/itm335feed041afb?pid=ACNGX8JPHYYXJGSX&lid=LSTACNGX8JPHYYXJGSXDUZR6X&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_20&otracker=browse&iid=10ac34b5-4af4-4100-a1aa-d13ee6c44a86.ACNGX8JPHYYXJGSX.SEARCH&ssid=yd80e3fvb40000001724715609163: name 'NoSuchElementException' is not defined


 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [06:02<00:31, 31.50s/it]

Error processing URL https://www.flipkart.com/lg-1-5-ton-5-star-split-inverter-ac-white/p/itm35753fe17cb49?pid=ACNGAYH9DGNFMYTX&lid=LSTACNGAYH9DGNFMYTXTP7I0R&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_23&otracker=browse&iid=10ac34b5-4af4-4100-a1aa-d13ee6c44a86.ACNGAYH9DGNFMYTX.SEARCH&ssid=yd80e3fvb40000001724715609163: name 'NoSuchElementException' is not defined


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [06:34<00:00, 30.31s/it]

Error processing URL https://www.flipkart.com/lg-1-5-ton-5-star-split-inverter-ac-white/p/itm1bd6c9db80037?pid=ACNGMVFTT9GYV3G6&lid=LSTACNGMVFTT9GYV3G6KIRULE&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_24&otracker=browse&iid=10ac34b5-4af4-4100-a1aa-d13ee6c44a86.ACNGMVFTT9GYV3G6.SEARCH&ssid=yd80e3fvb40000001724715609163: name 'NoSuchElementException' is not defined


In [6]:
df_LG_airconditioner = pd.DataFrame(all_reviews)
df_LG_airconditioner

,text,state,date,star,hp_data
0,Waste of money . AC not working till one month...,"Certified Buyer, Alwar","May, 2023",1,LG AI Convertible 6-in-1 Cooling 2024 Model 1....
1,I purchased this LG A. C. on 17 April 23..The ...,"Certified Buyer, Kalyan","Apr, 2023",1,LG AI Convertible 6-in-1 Cooling 2024 Model 1....
2,If you want to buy a white showpiece for your ...,"Certified Buyer, Madhyamgram","Apr, 2023",1,LG AI Convertible 6-in-1 Cooling 2024 Model 1....
3,Totally waste of money. company is not refund ...,"Certified Buyer, Charkhi Dadri","May, 2022",1,LG AI Convertible 6-in-1 Cooling 2024 Model 1....
4,"This model don't have 4 way swing, it's actual...","Certified Buyer, Erode",6 months ago,1,LG AI Convertible 6-in-1 Cooling 2024 Model 1....
...,...,...,...,...,...
1239,This is not performing as per requirement and ...,"Certified Buyer, Jajapur",4 months ago,1,LG 1.5 Ton 5 Star Split Inverter AC - White (...
1240,"Low cooling capacity and noise is very high, I...","Certified Buyer, Srikakulam District",4 months ago,1,LG 1.5 Ton 5 Star Split Inverter AC - White (...
1241,Too much cost,"Certified Buyer, Yerraguntla",4 months ago,1,LG 1.5 Ton 5 Star Split Inverter AC - White (...
1242,"Not getting cooling, Blue star is best.","Certified Buyer, Proddatur",4 months ago,1,LG 1.5 Ton 5 Star Split Inverter AC - White (...


In [7]:
df_LG_airconditioner.to_csv("df_LG_airconditioner.csv")